### **STEP-1**. Prepare for AI Solution Registration
&#x1F600; **등록 할 AI Contents 의 experimental_plan.yaml 를 alo/config/ 에 준비해 둡니다.**

&#x1F600; **가상 환경을 만들어 두고, ipykernel 을 제작해 둡니다.**     

- ALO 의 main.py 파일이 존재하는 위치에서 아래 명령어들을 실행합니다.
> conda create -n {name} python=3.10 \
> conda init bash \ 
> conda activate {name} \
> python main.py \
> pip install ipykernel \
> pip install requests \
> python -m ipykernel install --user --name {name} --display-name [ipykernel-name]

- 해당 jupyter notebook 에서 생성된 ipykernel 을 선택 합니다.     

<div style="margin: 40px">
<img src="./image/ipykernel.png" width="400">
</div>

----

### **STEP-2**. AI Solution 이름 선택     

#### **STEP-2-1**. AI Conductor 의 URI 입력
로그인 요청 및 시스템 담당에게 사용 가능한 URI 를 확인 합니다. 
- 고객지수플랫폼 Development 
> URI: "https://aic-kic.aidxlge.com/"
- 담당서버 테스트 환경       
> URI = "http://10.158.2.243:9999/"
----

> 사용자 입력 가변부

In [22]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
user_input ={
    # 로그인 정보: EP 정보로 입력해주세요
    'LOGIN_ID': 'ws.jang', # "cism-dev"
    'LOGIN_PW': '', # "cism-dev@com"
    
    # workspace 이름 
    'WORKSPACE_NAME': "magna-ws", # "cism-ws"
    
    # 시스템 URI
    'URI': "https://aic-web-kic.aidxlge.com/", #"http://10.158.2.243:9999/", 
    
    # ECR에 올라갈 컨테이너 URI TAG 
    'ECR_TAG': 'latest', 
    
    # scripts/creating_ai_solution/image/ 밑에 UI에 표시될 아이콘 이미지 파일 (ex. icon.png) 를 업로드 해주세요. 
    # 이후 해당 아이콘 파일 명을 아래에 기록 해주세요.
    'ICON_FILE': 'icon.png'
}
#----------------------------------------#

> 시스템 로그인

In [23]:
!pip install tabulate
# Generate RegisterUtils instance 
from register_utils import RegisterUtils
registerer = RegisterUtils(user_input)

URI_SCOPE:  magna-ws


In [24]:
# AI Conductor Login
registerer.login()


>> Success getting cookie from AI Conductor:
 {'access-token': 'FoLK8Q944JHAVXdo1yBQVQYYu2RSl2T8q1glPCqsYzQ'}

>> Success Login: {'result': 'OK'}


> AI Solution 이름 설정 (사용자 입력)

In [25]:
# AI Solution 이름 설정 (기존에 존재하는 중복 이름 허용X)
from IPython.display import display, HTML

# soltuion 이름 입력 받기 
display(HTML('<p style="font-size: 20px;"> < Create the name of AI Solution! > </p>'))
user_solution_name = input("- Enter the name you want to create: ")

# 기존에 존재하는 solution name과 겹치지 않는지 체크 
registerer.check_solution_name(user_solution_name)


- Enter the name you want to create:  aaa


[Success] Allowed name: << aaa >>


#### **STEE-2-2**. AI Solution Name 을 AI Conductor 에 등록합니다. 
: 이름이 등록되면 본 노트북 과정이 끝날 때까지 변경이 어려 울 수 있습니다. \
   변경이 필요할 경우 (2-1) 를 다시 실행하여 주시기 바랍니다. 
 

> 기존에 workspace에 존재하는 solution 이름을 조회하여, 사용자가 입력하는 이름이 유효한지 (고유한지) 확인합니다.

In [26]:
# 학습 용 solution_metadata.yaml 셋팅 
###################
pipeline = 'train'
###################
# check workspace (ECR, S3정보 셋팅까지 진행)
registerer.check_workspace() 
# 받아온 workspace 정보 기반으로 solution_metadata.yaml 셋팅 
registerer.set_yaml(pipeline = pipeline)

[{'id': '1b61dae3-e8f7-4b5f-9629-da0aedbb4a08', 'name': 'cism-ws', 'namespace': 'aic-ns-cism-ws', 'kubeflow_user': 'aic-user-cism-ws@aic.com', 's3_bucket_name': {'public': 's3-an2-cism-dev-aia', 'private': 's3-an2-cism-dev-cism'}, 'ecr_base_path': {'public': '042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/public/', 'private': '042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/cism/'}, 'execution_specs': [{'name': 'high', 'label': 'cism-ws-high', 'vcpu': 8, 'ram_gb': 32, 'gpu': 0}, {'name': 'standard', 'label': 'cism-ws-standard', 'vcpu': 2, 'ram_gb': 8, 'gpu': 0}], 'is_deleted': 0}, {'id': 'a0da0a72-3ce4-43e4-b510-b875896dcb35', 'name': 'magna-ws', 'namespace': 'aic-ns-magna-ws', 'kubeflow_user': 'aic-user-magna-ws@aic.com', 's3_bucket_name': {'public': 's3-an2-cism-dev-aia', 'private': 's3-an2-cism-dev-magna'}, 'ecr_base_path': {'public': '042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai

----

#### **STEP-3**. Train 용 Sample Data 등록

In [6]:
# data 등록 
# s3 접근확인
registerer.s3_access_check('data')

# s3 데이터 업로드
# 이전에 이미 해당 s3 경로에 존재하던 데이터는 지워집니다.
registerer.s3_upload_data()


[INFO] Start s3 access check without key file.

[INFO] AWS region: ap-northeast-2

[INFO] AWS S3 access check: OK

[INFO] Start uploading data into S3 from local folder:
 /home/jovyan/bolt/ai-advisor-sa/alo/input/train/

Success uploading into S3: 
s3-an2-cism-dev-cism/ai-solutions/magna-2312081506/v1/train/data/train/20230117_04_26_48_LTCO75645678010008_ETC2(0)_OK.csv

Success uploading into S3: 
s3-an2-cism-dev-cism/ai-solutions/magna-2312081506/v1/train/data/train/20230117_02_37_38_06Y8004090400000XP2405031012V5545_ETC2(5)_OTHERS(0).csv

Success uploading into S3: 
s3-an2-cism-dev-cism/ai-solutions/magna-2312081506/v1/train/data/train/20230120_05_40_05_06Y8004090400000XP2405031012V5545_ETC1(1)_OK.csv

Success uploading into S3: 
s3-an2-cism-dev-cism/ai-solutions/magna-2312081506/v1/train/data/train/20230904_15_38_57_LR23180A0031_ETC1(2)_OK.csv

Success uploading into S3: 
s3-an2-cism-dev-cism/ai-solutions/magna-2312081506/v1/train/data/train/20230904_15_40_00_LR23180A0031_ETC2(4)_O

In [7]:
# artifacts 등록 
# s3 접근확인
registerer.s3_access_check('artifacts')

# s3 model, artifacts 업로드 
# 이전에 이미 해당 s3 경로에 존재하던 object는 지워집니다.
registerer.s3_upload_artifacts()


[INFO] Start s3 access check without key file.

[INFO] AWS region: ap-northeast-2

[INFO] AWS S3 access check: OK

[INFO] Start uploading << train artifacts >> into S3 from local folder:
 /home/jovyan/bolt/ai-advisor-sa/alo/.temp_artifacts_dir/

Success uploading into S3: 
s3-an2-cism-dev-cism/ai-solutions/magna-2312081506/v1/train/artifacts/train_artifacts.tar.gz

Success updating solution_metadata.yaml - << artifact_uri >> info. / pipeline: train


In [8]:
# UI에 표시될 아이콘 이미지를 s3에 업로드합니다. 
registerer.s3_upload_icon()

Success uploading into S3 path: s3-an2-cism-dev-cism/icons/magna-2312081506/icon.png


----

#### **STEP-4**. Train Docker Container 제작

> ALO 작업 폴더를 현재 노트북 경로로 가져옵니다. 

In [9]:
# ALO 작업 폴더 가져오기 
registerer.set_alo()

[INFO] copy from " /home/jovyan/bolt/ai-advisor-sa/alo/main.py "  -->  " /home/jovyan/bolt/ai-advisor-sa/alo/scripts/creating_ai_solution/alo/ " 
[INFO] copy from " /home/jovyan/bolt/ai-advisor-sa/alo/src "  -->  " /home/jovyan/bolt/ai-advisor-sa/alo/scripts/creating_ai_solution/alo/src " 
[INFO] copy from " /home/jovyan/bolt/ai-advisor-sa/alo/config "  -->  " /home/jovyan/bolt/ai-advisor-sa/alo/scripts/creating_ai_solution/alo/config " 
[INFO] copy from " /home/jovyan/bolt/ai-advisor-sa/alo/assets "  -->  " /home/jovyan/bolt/ai-advisor-sa/alo/scripts/creating_ai_solution/alo/assets " 
[INFO] copy from " /home/jovyan/bolt/ai-advisor-sa/alo/alolib "  -->  " /home/jovyan/bolt/ai-advisor-sa/alo/scripts/creating_ai_solution/alo/alolib " 

 Success ALO directory setting.


> DOCKERFILE을 셋팅합니다. 

In [10]:
# DOCKERFILE setting
registerer.set_docker_contatiner()

Success DOCKERFILE setting. 
 - pipeline: train


> AWS ECR에 docker 등록을 위한 repository를 생성합니다. 

In [12]:
## ECR 등록
## docker login 실행 
#registerer.set_aws_ecr(docker=True)

## buildah login 실행 (docker in docker) - sudo 권한 필요 
tags = [
    "Key=Company,Value=LGE",
    "Key=Owner,Value=IC360",
    "Key=HQ,Value=CDO",
    "Key=Division,Value=CDO",
    "Key=Infra Region,Value=KIC",
    "Key=Service Mode,Value=DE",
    "Key=Cost Type,Value=COMPUTING",
    "Key=Project,Value=CIS",
    "Key=Sub Project,Value=CISM",
    "Key=System,Value=AIDX"
]
registerer.set_aws_ecr(docker=False, tags=tags) 

p1:  ['aws', 'ecr', 'get-login-password', '--region', 'ap-northeast-2']
[INFO] target AWS ECR url: 
042969618971.dkr.ecr.ap-northeast-2.amazonaws.com
p2:  ['buildah', 'login', '--username', 'AWS', '--password-stdin', '042969618971.dkr.ecr.ap-northeast-2.amazonaws.com']


time="2023-12-08T06:07:41Z" level=warning msg="error running newgidmap: exit status 1: newgidmap: write to gid_map failed: Operation not permitted\n"
time="2023-12-08T06:07:41Z" level=warning msg="falling back to single mapping"
time="2023-12-08T06:07:41Z" level=warning msg="error running newuidmap: exit status 1: newuidmap: write to uid_map failed: Operation not permitted\n"
time="2023-12-08T06:07:41Z" level=warning msg="falling back to single mapping"


[INFO] AWS ECR | docker login result: 
 Login Succeeded!

[INFO] Target AWS ECR repository: 
ecr-repo-an2-cism-dev/ai-solutions/magna/magna-2312081506/train/magna-2312081506

[INFO] AWS ECR create-repository response: 
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:ap-northeast-2:042969618971:repository/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-2312081506/train/magna-2312081506",
        "registryId": "042969618971",
        "repositoryName": "ecr-repo-an2-cism-dev/ai-solutions/magna/magna-2312081506/train/magna-2312081506",
        "repositoryUri": "042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-2312081506/train/magna-2312081506",
        "createdAt": "2023-12-08T06:07:42+00:00",
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": true
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}



> Docker Build

In [13]:
# docker build 
registerer.build_docker()

STEP 1/12: FROM public.ecr.aws/docker/library/python:3.10-slim-bullseye
STEP 2/12: RUN apt-get update
Get:1 http://deb.debian.org/debian bullseye InRelease [116 kB]
Get:2 http://deb.debian.org/debian-security bullseye-security InRelease [48.4 kB]
Get:3 http://deb.debian.org/debian bullseye-updates InRelease [44.1 kB]
Get:4 http://deb.debian.org/debian bullseye/main amd64 Packages [8062 kB]
Get:5 http://deb.debian.org/debian-security bullseye-security/main amd64 Packages [260 kB]
Get:6 http://deb.debian.org/debian bullseye-updates/main amd64 Packages [17.7 kB]
Fetched 8548 kB in 1s (7999 kB/s)
Reading package lists...
STEP 3/12: RUN apt-get install -y apt-utils
Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  apt-utils
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 439 kB of archives.
After this operation, 1192 kB of additional disk space will be used.
Get:1 http://deb.debian.org

debconf: delaying package configuration, since apt-utils is not installed


Fetched 439 kB in 0s (10.7 MB/s)
Selecting previously unselected package apt-utils.
(Reading database ... 7034 files and directories currently installed.)
Preparing to unpack .../apt-utils_2.2.4_amd64.deb ...
Unpacking apt-utils (2.2.4) ...
Setting up apt-utils (2.2.4) ...
STEP 4/12: RUN apt-get install -y --no-install-recommends          build-essential          wget          ca-certificates          git          gcc     && rm -rf /var/lib/apt/lists/*
Reading package lists...
Building dependency tree...
Reading state information...
ca-certificates is already the newest version (20210119).
The following additional packages will be installed:
  binutils binutils-common binutils-x86-64-linux-gnu bzip2 cpp cpp-10 dpkg-dev
  g++ g++-10 gcc-10 git-man libasan6 libatomic1 libbinutils libbrotli1
  libc-dev-bin libc6-dev libcc1-0 libcrypt-dev libctf-nobfd0 libctf0
  libcurl3-gnutls libdpkg-perl liberror-perl libgcc-10-dev libgdbm-compat4
  libgomp1 libisl23 libitm1 libldap-2.4-2 liblsan0 libmp

debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (Can't locate Term/ReadLine.pm in @INC (you may need to install the Term::ReadLine module) (@INC contains: /etc/perl /usr/local/lib/x86_64-linux-gnu/perl/5.32.1 /usr/local/share/perl/5.32.1 /usr/lib/x86_64-linux-gnu/perl5/5.32 /usr/share/perl5 /usr/lib/x86_64-linux-gnu/perl-base /usr/lib/x86_64-linux-gnu/perl/5.32 /usr/share/perl/5.32 /usr/local/lib/site_perl) at /usr/share/perl5/Debconf/FrontEnd/Readline.pm line 7, <> line 56.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Selecting previously unselected package libgdbm-compat4:amd64.
Preparing to unpack .../01-libgdbm-compat4_1.19-2_amd64.deb ...
Unpacking libgdbm-compat4:amd64 (1.19-2) ...
Selecting previously unselected package libperl5.32:amd64.
Preparing to unpack .../02-libperl5.32_5.32.1-4+deb11u2_amd64.deb ...
Unpacking libperl5.32:amd64 (5.32.1-4+deb11u2) ...
Selecting previously unselected package perl.
Preparing to unpack .../03-perl_5.32.1-4+deb11u2_amd64.deb ...
Unpacking perl (5.32.1-4+deb11u2) ...
Selecting previously unselected package bzip2.
Preparing to unpack .../04-bzip2_1.0.8-4_amd64.deb ...
Unpacking bzip2 (1.0.8-4) ...
Selecting previously unselected package libpsl5:amd64.
Preparing to unpack .../05-libpsl5_0.21.0-1.2_amd64.deb ...
Unpacking libpsl5:amd64 (0.21.0-1.2) ...
Selecting previously unselected package wget.
Preparing to unpack .../06-wget_1.21-1+deb11u1_amd64.deb ...
Unpacking wget (1.21-1+deb11u1) ...
Selecting previously unselected package xz-utils.
Preparing to unpack 

Getting image source signatures
Copying blob sha256:1b6fd3ad4ce602924fffb84437331a255e2a9463531a1bd92a15e9e3c4d11523
Copying blob sha256:12d601beac88c4f6011b0d711827f0a08383e99e300eebb128ee17ccbb935e10
Copying blob sha256:277bd6c0df20bd4b73a9dcb29dfbe5019209893d5a8078f3189fc801384555fe
Copying blob sha256:6b3d0b913afc500b93afb9ffbff3c144b25bc17d5780bccd3979ecb29ee1e472
Copying blob sha256:2f33dd8c934e3dbab052f70b9a53cd46dbc810a9fce8cd68609bff634d8a2f00
Copying blob sha256:aec798220f3723065fad4792eabc31a4e60ab623ace6333c59051e6fcfd0a4b7
Copying config sha256:25046a4924322f4ba801c0257ee7b2a115793c547b636dde43a8498ad9f095ef
Writing manifest to image destination
Storing signatures


--> 25046a49243
Successfully tagged 042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-2312081506/train/magna-2312081506:latest
25046a4924322f4ba801c0257ee7b2a115793c547b636dde43a8498ad9f095ef


> ECR에 docker image를 push 합니다. 

In [14]:
registerer.docker_push()

debug:  ['sudo', 'buildah', 'push', '042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-2312081506/train/magna-2312081506:latest']


Getting image source signatures
Copying blob sha256:6b3d0b913afc500b93afb9ffbff3c144b25bc17d5780bccd3979ecb29ee1e472
Copying blob sha256:2f33dd8c934e3dbab052f70b9a53cd46dbc810a9fce8cd68609bff634d8a2f00
Copying blob sha256:12d601beac88c4f6011b0d711827f0a08383e99e300eebb128ee17ccbb935e10
Copying blob sha256:277bd6c0df20bd4b73a9dcb29dfbe5019209893d5a8078f3189fc801384555fe
Copying blob sha256:1b6fd3ad4ce602924fffb84437331a255e2a9463531a1bd92a15e9e3c4d11523
Copying blob sha256:aec798220f3723065fad4792eabc31a4e60ab623ace6333c59051e6fcfd0a4b7
Copying config sha256:25046a4924322f4ba801c0257ee7b2a115793c547b636dde43a8498ad9f095ef


Removed login credentials for all registries


Writing manifest to image destination
Storing signatures


> solution_metadata.yaml에 container uri를 넣어줍니다.

In [15]:
registerer.set_container_uri() 

[INFO] Completes setting << container_uri >> in solution_metadata.yaml: 
042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-2312081506/train/magna-2312081506


#### **STEP-5**. Training 에 사용될 User Parameters 설정

> solution_metadata.yaml에 사용자 파라미터 및 artifacts 저장 경로를 넣어줍니다. 

In [16]:
## user parameter 입력
registerer.set_candidate_parameters() # todo: selected params?
## artifact 저장 경로 지정
registerer.set_artifacts_uri()


[{self.pipeline}] Success updating << candidate_parameters >> in the solution_metadata.yaml
[INFO] Completes setting << artifact_uri >> in solution_metadata.yaml: 
s3://s3-an2-cism-dev-cism/ai-solutions/magna-2312081506/v1/train/artifacts/


#### **STEP-6**. Training 에 사용될 Cloud resource 선택

> 컴퓨팅 자원을 선택합니다. (default: standard)

In [17]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#

train_resource = 'standard'

#----------------------------------------#

# 클라우드 컴퓨팅 리소스 선택 
registerer.set_resource(train_resource)


[train] Success updating << resource >> in the solution_metadata.yaml


#### **STEP-7**. Inference 용 Sample Data 등록

In [18]:
###################
pipeline = "inference"
###################
# inference pipeline을 solution_metadata.yaml에 추가하고, 현재 registerer의 pipeline type을 inference로 변경 
registerer.append_pipeline(pipeline)
# s3 데이터 업로드
# 이전에 있던 데이터는 지워집니다
registerer.s3_access_check('data')
registerer.s3_upload_data()


<< solution_metadata.yaml >> updated. - appended pipeline: inference

[INFO] Start s3 access check without key file.

[INFO] AWS region: ap-northeast-2

[INFO] AWS S3 access check: OK

[INFO] Start uploading << data >> into S3 from local folder:
 /home/jovyan/bolt/ai-advisor-sa/alo/input/inference/

Success uploading into S3: 
s3-an2-cism-dev-cism/ai-solutions/magna-2312081506/v1/inference/data/test/20231006_16_57_06_LR23235A0040_ETC1(2)_OK.csv

Success updating solution_metadata.yaml - << dataset_uri >> info. / pipeline: inference


In [19]:
registerer.s3_access_check('artifacts')
registerer.s3_upload_artifacts()


[INFO] Start s3 access check without key file.

[INFO] AWS region: ap-northeast-2

[INFO] AWS S3 access check: OK

[INFO] Start uploading << inference artifacts >> into S3 from local folder:
 /home/jovyan/bolt/ai-advisor-sa/alo/.temp_artifacts_dir/

Success uploading into S3: 
s3-an2-cism-dev-cism/ai-solutions/magna-2312081506/v1/inference/artifacts/inference_artifacts.tar.gz

Success updating solution_metadata.yaml - << artifact_uri >> info. / pipeline: inference

[INFO] Start uploading << model >> into S3 from local folder:
 /home/jovyan/bolt/ai-advisor-sa/alo/.temp_model_dir/

Success uploading into S3: 
s3-an2-cism-dev-cism/ai-solutions/magna-2312081506/v1/train/artifacts/model.tar.gz

Success updating solution_metadata.yaml - << model_uri >> info. / pipeline: inference


#### **STEP-8**. Inference 용 Docker Container 제작

> AWS ECR에 docker 등록을 위한 repository를 생성합니다. 

In [22]:
## ECR 등록
## docker login 실행 
# registerer.set_aws_ecr(docker=True)

## buildah login 실행 (docker in docker) 
tags = [
    "Key=Company,Value=LGE",
    "Key=Owner,Value=IC360",
    "Key=HQ,Value=CDO",
    "Key=Division,Value=CDO",
    "Key=Infra Region,Value=KIC",
    "Key=Service Mode,Value=DE",
    "Key=Cost Type,Value=COMPUTING",
    "Key=Project,Value=CIS",
    "Key=Sub Project,Value=CISM",
    "Key=System,Value=AIDX"
]
registerer.set_aws_ecr(docker=False, tags=tags) 

p1:  ['aws', 'ecr', 'get-login-password', '--region', 'ap-northeast-2']
[INFO] target AWS ECR url: 
042969618971.dkr.ecr.ap-northeast-2.amazonaws.com
p2:  ['buildah', 'login', '--username', 'AWS', '--password-stdin', '042969618971.dkr.ecr.ap-northeast-2.amazonaws.com']


time="2023-12-08T06:09:41Z" level=warning msg="error running newgidmap: exit status 1: newgidmap: write to gid_map failed: Operation not permitted\n"
time="2023-12-08T06:09:41Z" level=warning msg="falling back to single mapping"
time="2023-12-08T06:09:41Z" level=warning msg="error running newuidmap: exit status 1: newuidmap: write to uid_map failed: Operation not permitted\n"
time="2023-12-08T06:09:41Z" level=warning msg="falling back to single mapping"


[INFO] AWS ECR | docker login result: 
 Login Succeeded!

[INFO] Target AWS ECR repository: 
ecr-repo-an2-cism-dev/ai-solutions/magna/magna-2312081506/inference/magna-2312081506

[INFO] AWS ECR create-repository response: 
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:ap-northeast-2:042969618971:repository/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-2312081506/inference/magna-2312081506",
        "registryId": "042969618971",
        "repositoryName": "ecr-repo-an2-cism-dev/ai-solutions/magna/magna-2312081506/inference/magna-2312081506",
        "repositoryUri": "042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-2312081506/inference/magna-2312081506",
        "createdAt": "2023-12-08T06:09:43+00:00",
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": true
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}



In [23]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#

pre_existing_docker = '10.157.52.219:5000/alo:2.1.2_amd_mql_1.0'

#----------------------------------------#


registerer.rename_docker(pre_existing_docker)

[Success] done renaming docker: 
 10.157.52.219:5000/alo:2.1.2_amd_mql_1.0 --> 042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-2312081506/inference/magna-2312081506:latest


> Build docker

> AWS ECR에 docker 이미지를 push 합니다. 

In [24]:
registerer.docker_push()

debug:  ['sudo', 'buildah', 'push', '042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-2312081506/inference/magna-2312081506:latest']
Removed login credentials for all registries


error pushing image "042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-2312081506/inference/magna-2312081506:latest" to "docker://042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-2312081506/inference/magna-2312081506:latest": 042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-2312081506/inference/magna-2312081506:latest: image not known
time="2023-12-08T06:10:35Z" level=error msg="exit status 125"


> solution_metadata.yaml에 container uri를 넣어줍니다.

In [26]:
registerer.set_container_uri() # uri도 그냥 입력되게 수정

[INFO] Completes setting << container_uri >> in solution_metadata.yaml: 
042969618971.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-cism-dev/ai-solutions/magna/magna-2312081506/inference/magna-2312081506


#### **STEP-9**. Inference 용 User Parameters 제작

> solution_metadata.yaml에 사용자 파라미터, artifacts 경로 등을 넣어줍니다.

In [27]:
registerer.set_candidate_parameters()
registerer.set_artifacts_uri()


[{self.pipeline}] Success updating << candidate_parameters >> in the solution_metadata.yaml
[INFO] Completes setting << artifact_uri >> in solution_metadata.yaml: 
s3://s3-an2-cism-dev-cism/ai-solutions/magna-2312081506/v1/inference/artifacts/


> solution_metadata.yaml에 model 경로를 넣어줍니다. (추론 시에만 필요합니다.)

In [28]:
registerer.set_model_uri() # 주의: model은 train artifacts 경로에 존재

[INFO] Completes setting << model_uri >> in solution_metadata.yaml: 
s3://s3-an2-cism-dev-cism/ai-solutions/magna-2312081506/v1/train/artifacts/


#### **STEP-10**. Inference 용 Resource 선택

> 컴퓨팅 자원을 선택합니다.

In [29]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#

inference_resource = 'standard'

#----------------------------------------#

registerer.set_resource(inference_resource)


[inference] Success updating << resource >> in the solution_metadata.yaml


#### **STEP-11**. Description 추가

> solution_metadata.yaml에 description을 추가합니다.

In [30]:
# [임시] 수정필요 
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
user_description ={
    'overview': "AI Advisor Test",

    'input_data': "Test input s3 bucket",
    
    'output_data': "Test output s3 bucket",

    'user_parameters': "Test params",
    
    'algorithm': "ALO"
}
#----------------------------------------#

registerer.set_description(user_description)


<< solution_metadata.yaml >> updated. 
- solution metadata description:
 {'icon': 's3://s3-an2-cism-dev-cism/icons/magna-2312081506/icon.png', 'title': 'magna-2312081506', 'overview': 'AI Advisor Test', 'input_data': 's3-an2-cism-dev-cismTest input s3 bucket', 'output_data': 's3-an2-cism-dev-cismTest input s3 bucket', 'user_parameters': 'Test params', 'algorithm': 'ALO'}


#### **STEP-12**. AI Solution 등록 !!

> AI Solution 등록을 진행합니다.

In [31]:
# wrangler 정보 등록 
# [임시] 변경 예정
registerer.set_wrangler()

In [32]:
# Edge관련 정보 등록  
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
edgeconductor_interface = {
            'support_labeling': True,
            
            'inference_result_datatype': 'image', # 'image'
            
            'train_datatype': 'table' # 'image'
        }
#----------------------------------------#
registerer.set_edge(edgeconductor_interface)

In [33]:
registerer.register_solution()

[INFO] AI solution register response: 
 {'detail': 'Invalid credentials'}


> Solution instance 등록

In [33]:
registerer.register_solution_instance()


[INFO] AI solution interface information: 
 {'name': 'abc', 'solution_version_id': 'f12315dc-7e7a-4371-a26b-46f3bec7fe73', 'workspace_name': 'magna-ws'}

[INFO] AI solution instance register response: 
 {'id': '86e96de6-ebd7-423e-9313-10122b6be745', 'name': 'abc', 'workspace_id': 'a0da0a72-3ce4-43e4-b510-b875896dcb35', 'workspace_name': 'magna-ws', 'solution_version_id': 'f12315dc-7e7a-4371-a26b-46f3bec7fe73', 'solution_version': 1, 'metadata_dict': {'version': 1, 'name': 'abc', 'description': {'icon': 's3://s3-an2-hyunsoo-dev-aia/icons/abc/icon.png', 'title': 'abc', 'overview': 'AI Advisor Test', 'input_data': 's3-an2-hyunsoo-dev-aiaTest input s3 bucket', 'output_data': 's3-an2-hyunsoo-dev-aiaTest input s3 bucket', 'user_parameters': 'Test params', 'algorithm': 'ALO'}, 'pipeline': [{'type': 'train', 'dataset_uri': ['s3://s3-an2-hyunsoo-dev-aia/ai-solutions/abc/v1/train/data/'], 'container_uri': '086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/ai-solutions/p